In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()#get(name="udacity-project")#, resource_group='aml-vm-174174')
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-175073
Azure region: southcentralus
Subscription id: 510b94ba-e453-4417-988b-fbdc37b55ca7
Resource group: aml-quickstarts-175073


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your CPU cluster
cluster_name = "cpucluster4" #was cpu_cluster_name

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name) #was cpu_cluster
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # To use a different region for the compute, add a location='<region>' parameter
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

Found existing cluster, use it.

Running


In [11]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os

# Specify parameter sampler
experiment=Experiment(ws,'quick-starts-ws-175073')
ps = RandomParameterSampling( {
       "--C": choice (0.01,0.1,1),
       "--max_iter": choice(20,40,60,100,150,200)#choice(16,32,64,128) 
   })
# ({"learning_rate":uniform(0.05,0.1),"batch_size":uniform(16,32,64,128)})### YOUR CODE HERE ###

# Specify a Policy
policy = BanditPolicy(slack_factor=0.1,evaluation_interval=2)### YOUR CODE HERE ###

if "training" not in os.listdir():
    os.mkdir("./training")

#Setup Environment for your training run
from azureml.core.environment import Environment

sklearn_run=Environment.from_conda_specification(name='sklearn-env',file_path='conda_dependencies.yml')

#Create a ScriptRunConfig Object to specify the configuration details of your trianing job
from azureml.core import ScriptRunConfig
est=ScriptRunConfig(source_directory="./training", script='train.py',compute_target=compute_target)

# Create a SKLearn estimator for use with train.py
# est = SKLearn(source_directory='./training', entry_script='train.py', compute_target=compute_target,vm_size='STANDARD_D2_V2')### YOUR CODE HERE ###
#train.py should be inside ./training folder! ##!!!!cpu_cluster_name
# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.

hyperdrive_config = HyperDriveConfig(run_config=est,hyperparameter_sampling=ps, primary_metric_name='Accuracy',primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,max_total_runs=4,policy=policy) #, estimator=est

### YOUR CODE HERE ### #primary_metric_name="AUC_weighted" results in error!! Accuracy
##accuracy can be used as well

In [12]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
hyperdrive_run=exp.submit(config=hyperdrive_config) #was pipeline_run
RunDetails(hyperdrive_run).show()
# pipeline_run.get_status()
hyperdrive_run.wait_for_completion(show_output=True) #show_output=True don't include otherwise error

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_8788b67b-b0f7-4e6e-961c-bbc38416b915
Web View: https://ml.azure.com/runs/HD_8788b67b-b0f7-4e6e-961c-bbc38416b915?wsid=/subscriptions/510b94ba-e453-4417-988b-fbdc37b55ca7/resourcegroups/aml-quickstarts-175073/workspaces/quick-starts-ws-175073&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

"<START>[2022-01-13T15:04:19.964756][API][INFO]Experiment created<END>\n""<START>[2022-01-13T15:04:21.017492][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n"


In [5]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
best_run=hyperdrive_run.get_best_run_by_primary_metric()
# gives error-suggests that the best_run has the value equivalence of null in other languages, which means it des not contain 
#anything. This means that the there was no result from pipeline_run.get_best_run_by_primary_metric() as used in the notebook
#to retrieve the best run, and thus nothing was assigned the best_run object. The question then would be: did the
#training complete successfully?

# print('fiel names {}'.format(best_run.get_file_names()))

model = best_run.register_model(model_name = 'best_model', model_path = 'outputs/model.joblib')
#, tags={"Method":"HyperDrive"}, properties={"Accuracy":best_run.get_metrics()["Accuracy"]})
#https://github.com/Azure/MachineLearningNotebooks/issues/269
#!!!!

AttributeError: 'NoneType' object has no attribute 'register_model'

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
url="https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds=TabularDatasetFactory.from_delimited_files(path=url)

In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)### YOUR DATA OBJECT HERE ###)

In [ ]:
from azureml.train.automl import AutoMLConfig
from sklearn.model_selection import train_test_split

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.


x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)


automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task="classification",
    primary_metric="Accuracy", #AUC_weighted
    training_data=ds,

    label_column_name='y',
    n_cross_validations=5)

In [ ]:
# Submit your automl run
run=exp.submit(config=automl_config,show_output=True)
### YOUR CODE HERE ###

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
run.get_metrics()

import onnxmltools

best_run, fitted_model = run.get_output()
#https://docs.microsoft.com/en-us/azure/machine-learning/tutorial-auto-train-models

onnxmltools.utils.save_model(onnx_model, './outputs/best_automl.onnx')

In [ ]:
cpu_cluster.delete()
#https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.compute.computetarget?view=azure-ml-py#delete--